In [8]:
import Pkg; Pkg.activate(".")
import MeshCat as mc 
using LinearAlgebra

  Activating environment at `~/devel/recitations-2024/Untitled Folder/Project.toml`


In [37]:
# getting rotation matrix from axis-angle vectors
function skew(v)
    [0 -v[3] v[2]; v[3] 0 -v[1]; -v[2] v[1] 0]
end
function dcm_from_phi(ϕ)
    # rotation matrix from axis angle 
    # phi = r * θ, where theta is the axis of rotation (unit vector)
    # and θ = the angle of rotation
    theta = norm(ϕ)
    r = (abs(theta)>1e-12) ? ϕ/theta : zeros(3)
    Q = (I + sin(theta) * skew(r) + (1.0 - cos(theta)) *
    skew(r) * skew(r))
    return Q 
end

dcm_from_phi (generic function with 1 method)

In [38]:
# create a cylinder from a length and a radius 
function create_cyl(vis, len, R)
    # x axis aligned 
    a = [-len/2, 0, 0]
    b = [+len/2, 0, 0]
    cyl = mc.Cylinder(mc.Point(a...), mc.Point(b...), R)
    material = mc.MeshPhongMaterial(color=mc.RGBA(1.0, 0.0, 0.0, 1.0))
    mc.setobject!(vis[:rope], cyl, material)
end

create_cyl (generic function with 1 method)

In [39]:
function get_dcm_from_ends(a, b)
    # get a rotation matrix from ends of a cylinder 
    nx = normalize(b - a)
    e = [1.242,4.242,-3.424]  
    # taking a cross product with random e gives us a new vector 
    # that is orthogonal to both nx and e. We make this ny 
    ny = normalize(cross(nx, e))
    nz = normalize(cross(nx, ny))
    
    N_Q_B = [nx ny nz]
end

get_dcm_from_ends (generic function with 1 method)

In [40]:
let 
    vis = mc.Visualizer()
    
    # NOTICE HOW I USE [:<OBJECT>][:base] WHEN DEFINING THE INITIAL OBJ OFFSETS 
    # THEN WHEN I DO ANIMATION, I JUST USE [:<OBJECT>]
    
    # dragon details (play with these to see how they change)
    dragon_obj = mc.MeshFileGeometry(joinpath(@__DIR__,"dragon.obj"))
    dragon_material = mc.MeshPhongMaterial(color=mc.RGBA(0.6, 0.6, 1.0, 0.7))
    dragon_r_offset = [0,0,-.34]
    dragon_scale = 0.002 
    dragon_Q_offset = dcm_from_phi(pi/2*[1,0,0])
    dragon_offset = mc.compose(
        mc.Translation(dragon_r_offset),
        mc.LinearMap(dragon_scale * dragon_Q_offset)
    )
    
    # create first dragon 
    mc.setobject!(vis[:dragon][:base], dragon_obj, dragon_material) # NOTE :base
    mc.settransform!(vis[:dragon][:base], dragon_offset)            # NOTE :base
    
    # quadrotor details (play with these to see how they change)
    quad_obj = mc.MeshFileGeometry(joinpath(@__DIR__,"quadrotor.obj"))
    quad_material = mc.MeshPhongMaterial(color=mc.RGBA(1.0, 0.6, 0.6, 0.7))
    quad_r_offset = [0,0,0]
    quad_scale = 2.5 
    quad_Q_offset = dcm_from_phi([0,0,0])
    quad_offset = mc.compose(
        mc.Translation(quad_r_offset),
        mc.LinearMap(quad_scale * quad_Q_offset)
    )
    
    # create quadrotor 
    mc.setobject!(vis[:quad][:base], quad_obj, quad_material)       # NOTE :base
    mc.settransform!(vis[:quad][:base], quad_offset)                # NOTE :base
     
    # create rope between them
    rope_length = 5.0 
    create_cyl(vis, rope_length, 0.05)
    
    # animation 
    tf = 10.0 
    N = 100 
    t_vec = range(0.0, tf, length=N)
    dt = t_vec[2] - t_vec[1]
    
    # position of dragon 
    r_dragon = [[cos(t), sin(t), sin(t)] for t in t_vec]
    r_quad = [r + rope_length*normalize([1 + cos(t), -1 + cos(t), -.3 + cos(t)]) for (r,t) in zip(r_dragon,t_vec)]
    
    for (a,b) in zip(r_dragon, r_quad)
        @assert norm(a - b) ≈ rope_length
    end
    
    # rope attitudes 
    rope_Qs = [get_dcm_from_ends(a, b) for (a,b) in zip(r_dragon, r_quad)]
    rope_rs = [(a + b) / 2 for (a,b) in zip(r_dragon, r_quad)]
    
    # make up attitudes for dragon and quad
    dragon_Qs = [dcm_from_phi(θ * [1, .2, .3]) for θ in range(0.0, pi/2, length=N)]
    quad_Qs = [dcm_from_phi(θ * [-.1, 1, -.3]) for θ in range(0.0, pi/2, length=N)]

    # animate 
    anim = mc.Animation(floor(Int,1/dt))
        for k = 1:N
            mc.atframe(anim, k) do
                # NOTE no :base
                mc.settransform!(vis[:dragon], mc.compose(  
                    mc.Translation(r_dragon[k]),
                    mc.LinearMap(dragon_Qs[k])
                ))
                # NOTE no :base
                mc.settransform!(vis[:quad], mc.compose(
                    mc.Translation(r_quad[k]),
                    mc.LinearMap(quad_Qs[k])
                ))
            
                mc.settransform!(vis[:rope], mc.compose(
                    mc.Translation(rope_rs[k]),
                    mc.LinearMap(rope_Qs[k])
                ))
            end
        end
    mc.setanimation!(vis, anim)
    mc.render(vis)
end

[ Info: Listening on: 127.0.0.1:8723, thread id: 1
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8723


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("quad" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x71, 0x75, 0x61, 0x64, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x71, 0x75, 0x61, 0x64, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "dragon" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x61, 0x67, 0x6f, 0x6e, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x61, 0x67, 0x6f, 0x6e, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "rope" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x70, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8723, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8723), "127.0.0.1", "8723", nothing, Sockets.TCPServer(RawFD(111) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x0000000173ac9ee0, ReentrantLock(nothing, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), 0))))